In [ ]:
import math as m
import numpy as np
import random as r
import matplotlib.pyplot as plt
import sys

In [ ]:
import torch
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR

In [ ]:
from nflows.flows.base import Flow
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedPiecewiseRationalQuadraticAutoregressiveTransform
from nflows.transforms.autoregressive import MaskedPiecewiseQuadraticAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation
from nflows.transforms.permutations import RandomPermutation
from nflows.transforms.splines.rational_quadratic import rational_quadratic_spline
from torch.utils.tensorboard import SummaryWriter

In [ ]:
import subprocess
import time
import os
from copy import deepcopy
import math as m
import gc

## Tensorboard writer for loss logging

In [ ]:
writer = SummaryWriter()

## GPU/CPU selection

In [ ]:
#device = torch.device("cuda:0")
device = torch.device("cpu")

## Hyperparameters

In [ ]:
n_RQS_knots = 10   # Number of knots in RQS transform
n_made_layers = 3  # Number of layers in every made network
n_made_units = 500 # Number of units in every layer of the made network
n_flow_layers = 12 # Number of layers in the flow

batch_size = 1024
n_epochs = 800
adam_lr = 0.001   # Learning rate for the ADAM optimizer (default: 0.001)

n_train = int(1e6) # This is missing the required statistical factor to account for negative weights
n_test = int(1e5)

## Load the training data

In [ ]:
samples = np.genfromtxt("data/negative_weight_samples.csv", delimiter=',')
weights = np.genfromtxt("data/negative_weight_weights.csv", delimiter=',')

## Find the fraction of negative events and the required statistical factor

In [ ]:
f = np.sum(weights < 0)/len(weights)
c = (1-2*f)**-2
print("Fraction of negative events", f)
print("Statistical factor", c)
if (n_train + n_test)*c > samples.shape[0]:
    print("Not enough training data")

## Split to a train and test set

In [ ]:
n_train_with_stats = int(n_train*c)
n_test_with_stats = int(n_test*c)

train_samples = torch.tensor(samples[:n_train_with_stats], dtype=torch.float32, device=device)
train_weights = torch.tensor(weights[:n_train_with_stats], dtype=torch.float32, device=device)
test_samples = torch.tensor(samples[n_train_with_stats:n_train_with_stats+n_test_with_stats], dtype=torch.float32, device=device)
test_weights = torch.tensor(weights[n_train_with_stats:n_train_with_stats+n_test_with_stats], dtype=torch.float32, device=device)

del samples
del weights
gc.collect()

## Set up the flow

In [ ]:
event_dim = train_samples.shape[1]
base_dist = BoxUniform(torch.zeros(event_dim), torch.ones(event_dim))

transforms = []
for _ in range(n_flow_layers):
    transforms.append(RandomPermutation(features=event_dim))
    transforms.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        features=event_dim, 
        hidden_features=n_made_units,
        num_bins=n_RQS_knots,
        num_blocks=n_made_layers-1,
        tails="constrained",
        use_residual_blocks=False
    ))
transform = CompositeTransform(transforms)

flow = Flow(transform, base_dist).to(device)
optimizer = optim.Adam(flow.parameters(), lr=adam_lr)

scheduler = MultiStepLR(optimizer, milestones=[350, 425, 500, 575, 650, 725, 800], gamma=0.5)

## Training

In [ ]:
data_size = train_samples.shape[0]
n_batches = m.ceil(data_size/batch_size)

data_size_validation = test_samples.shape[0]
n_batches_validate = m.ceil(data_size_validation/batch_size)

best_loss = np.inf
for epoch in range(n_epochs):
    scheduler.step()
    
    permutation = torch.randperm(data_size, device=device)    

    # Loop over batches
    cum_loss = 0
    for batch in range(n_batches):
        # Set up the batch
        batch_begin = batch*batch_size
        batch_end   = min( (batch+1)*batch_size, data_size-1 )
        indices = permutation[batch_begin:batch_end]
        samples_batch = train_samples[indices]
        weights_batch = train_weights[indices]
        
        # Take a step
        optimizer.zero_grad()
        loss = -(flow.log_prob(inputs=samples_batch)*weights_batch).mean()
        loss.backward()
        optimizer.step()

        # Compute cumulative loss
        cum_loss = (cum_loss*batch + loss.item())/(batch+1)

        if batch%25 == 0:
            print("epoch = ", epoch, "batch = ", batch, "/", n_batches, "loss = ", cum_loss)
    
    # Compute validation loss
    validation_loss = 0
    for batch in range(n_batches_validate):
        batch_begin = batch*batch_size
        batch_end = min( (batch+1)*batch_size, data_size_validation-1 )
        samples_batch = testing_samples[batch_begin:batch_end]
        weights_batch = testing_weights[batch_begin:batch_end]
    
        with torch.no_grad():
            validation_loss = (validation_loss*batch - (flow.log_prob(samples_batch)*weights_batch).mean())/(batch+1)
    
    print("Validation loss = ", validation_loss.item())
    
    writer.add_scalar("Loss_train", cum_loss, epoch)
    writer.add_scalar("Loss_test", validation_loss, epoch)
    
    if validation_loss < best_loss:
        torch.save(flow, "best_flow_model.pt")
        best_loss = validation_loss

torch.save(flow, "final_flow_model.pt")